In [1]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

In [2]:
df=pd.read_csv("weather_train.csv")
df.head()

,Unnamed: 0,DATE (YYYY/MM/DD),MST,Global CMP22 (vent/cor) [W/m^2],Direct sNIP [W/m^2],Azimuth Angle [degrees],Tower Dry Bulb Temp [deg C],Tower Wet Bulb Temp [deg C],Tower Dew Point Temp [deg C],Tower RH [%],Peak Wind Speed @ 6ft [m/s],Avg Wind Direction @ 6ft [deg from N],Station Pressure [mBar],Precipitation (Accumulated) [mm],Moisture,Albedo (CMP11),Cloud Type,Solar Zenith Angle,Snow Depth [cm]
0,0,2020/02/22,04:11,-1.12724,-0.365968,77.66760,8.21,0.388,-12.412,19.20,1.20,300.10,817.297,0.000,0.0,0.0000,0,50.16,9.460
1,1,2020/07/25,20:10,-1.04030,0.093816,304.73416,20.14,14.855,11.955,59.29,4.45,262.40,820.184,5.334,0.0,0.0000,7,57.39,1.121
2,2,2020/09/13,04:03,-1.29707,-0.251820,68.24852,14.08,6.560,-0.940,35.17,2.45,280.20,822.653,0.000,0.0,0.0000,8,65.63,-0.050
3,3,2020/03/08,15:25,79.20020,-0.014836,237.11811,15.03,5.725,-4.575,24.36,3.70,23.07,809.641,0.000,0.0,0.1587,6,57.08,0.734
4,4,2020/11/17,19:33,-1.61485,-0.459201,271.20406,12.66,3.180,-10.220,17.37,2.70,283.80,818.717,0.000,0.0,0.0000,0,128.59,0.013


In [3]:
df.isnull().sum()

Unnamed: 0                               0
DATE (YYYY/MM/DD)                        0
MST                                      0
Global CMP22 (vent/cor) [W/m^2]          0
Direct sNIP [W/m^2]                      0
Azimuth Angle [degrees]                  0
Tower Dry Bulb Temp [deg C]              0
Tower Wet Bulb Temp [deg C]              0
Tower Dew Point Temp [deg C]             0
Tower RH [%]                             0
Peak Wind Speed @ 6ft [m/s]              0
Avg Wind Direction @ 6ft [deg from N]    0
Station Pressure [mBar]                  0
Precipitation (Accumulated) [mm]         0
Moisture                                 0
Albedo (CMP11)                           0
Cloud Type                               0
Solar Zenith Angle                       0
Snow Depth [cm]                          0
dtype: int64

In [4]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
dfle = df
dfle['DATE (YYYY/MM/DD)'] = le.fit_transform(dfle['DATE (YYYY/MM/DD)'])
dfle['MST'] = le.fit_transform(dfle['MST'])
dfle

,Unnamed: 0,DATE (YYYY/MM/DD),MST,Global CMP22 (vent/cor) [W/m^2],Direct sNIP [W/m^2],Azimuth Angle [degrees],Tower Dry Bulb Temp [deg C],Tower Wet Bulb Temp [deg C],Tower Dew Point Temp [deg C],Tower RH [%],Peak Wind Speed @ 6ft [m/s],Avg Wind Direction @ 6ft [deg from N],Station Pressure [mBar],Precipitation (Accumulated) [mm],Moisture,Albedo (CMP11),Cloud Type,Solar Zenith Angle,Snow Depth [cm]
0,0,52,251,-1.12724,-0.365968,77.66760,8.210,0.388,-12.412,19.20,1.20,300.10,817.297,0.000,0.0,0.0000,0,50.16,9.460
1,1,206,1210,-1.04030,0.093816,304.73416,20.140,14.855,11.955,59.29,4.45,262.40,820.184,5.334,0.0,0.0000,7,57.39,1.121
2,2,256,243,-1.29707,-0.251820,68.24852,14.080,6.560,-0.940,35.17,2.45,280.20,822.653,0.000,0.0,0.0000,8,65.63,-0.050
3,3,67,925,79.20020,-0.014836,237.11811,15.030,5.725,-4.575,24.36,3.70,23.07,809.641,0.000,0.0,0.1587,6,57.08,0.734
4,4,321,1173,-1.61485,-0.459201,271.20406,12.660,3.180,-10.220,17.37,2.70,283.80,818.717,0.000,0.0,0.0000,0,128.59,0.013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,14995,110,1127,3.69585,-0.553937,286.76511,13.280,5.410,-3.090,30.95,1.20,35.00,815.183,0.000,0.0,0.0000,0,91.76,0.954
14996,14996,285,540,515.44400,912.832000,131.12291,16.980,5.677,-8.323,15.55,8.20,302.50,817.411,0.000,0.0,0.2326,0,44.57,-0.277
14997,14997,365,560,286.39400,872.952000,140.97204,3.642,-1.619,-10.419,31.52,0.00,0.00,812.671,0.000,0.0,0.5277,3,82.88,2.426
14998,14998,157,25,-1.38846,0.064190,6.62879,23.580,11.528,1.928,24.13,3.45,242.80,813.969,0.000,0.0,0.0000,0,133.73,0.777


In [5]:
x_train = dfle.iloc[:,1:18].values
y_train = dfle.iloc[:,18:19]

In [6]:
x_train

array([[ 5.20000e+01,  2.51000e+02, -1.12724e+00, ...,  0.00000e+00,
         0.00000e+00,  5.01600e+01],
       [ 2.06000e+02,  1.21000e+03, -1.04030e+00, ...,  0.00000e+00,
         7.00000e+00,  5.73900e+01],
       [ 2.56000e+02,  2.43000e+02, -1.29707e+00, ...,  0.00000e+00,
         8.00000e+00,  6.56300e+01],
       ...,
       [ 3.65000e+02,  5.60000e+02,  2.86394e+02, ...,  5.27700e-01,
         3.00000e+00,  8.28800e+01],
       [ 1.57000e+02,  2.50000e+01, -1.38846e+00, ...,  0.00000e+00,
         0.00000e+00,  1.33730e+02],
       [ 2.47000e+02,  9.57000e+02,  4.35381e+02, ...,  2.18100e-01,
         1.00000e+00,  5.40600e+01]])

In [7]:
y_train

,Snow Depth [cm]
0,9.460
1,1.121
2,-0.050
3,0.734
4,0.013
...,...
14995,0.954
14996,-0.277
14997,2.426
14998,0.777


In [8]:
df_xtest = pd.read_csv("weather_test.csv")
dfle_xtest = df_xtest
dfle_xtest['DATE (YYYY/MM/DD)'] = le.fit_transform(dfle_xtest['DATE (YYYY/MM/DD)'])
dfle_xtest['MST'] = le.fit_transform(dfle_xtest['MST'])
dfle_xtest

,Unnamed: 0,DATE (YYYY/MM/DD),MST,Global CMP22 (vent/cor) [W/m^2],Direct sNIP [W/m^2],Azimuth Angle [degrees],Tower Dry Bulb Temp [deg C],Tower Wet Bulb Temp [deg C],Tower Dew Point Temp [deg C],Tower RH [%],Peak Wind Speed @ 6ft [m/s],Avg Wind Direction @ 6ft [deg from N],Station Pressure [mBar],Precipitation (Accumulated) [mm],Moisture,Albedo (CMP11),Cloud Type,Solar Zenith Angle
0,0,255,974,-0.821587,-0.197822,270.39668,0.344,-3.269,-9.469,43.43,1.95,300.70,817.643,0.000,0.0,0.0000,7,163.53
1,1,47,203,-0.464099,0.039564,83.18453,-15.420,-16.249,-19.649,67.07,1.20,308.30,809.469,0.000,0.0,0.0000,0,84.09
2,2,63,119,-1.133330,-0.009875,35.33772,22.380,11.522,3.122,28.25,1.70,214.00,813.392,0.000,0.0,0.0000,7,109.24
3,3,110,694,849.620000,946.128000,244.34737,14.240,6.585,-1.215,34.02,3.45,45.53,821.467,0.000,0.0,0.1844,7,156.80
4,4,127,765,44.193400,0.488831,230.89280,-5.580,-6.169,-7.269,86.40,1.20,15.69,812.324,0.000,1.0,0.6648,7,154.89
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2401,2401,185,1040,-1.444170,-0.360447,292.96778,9.820,3.214,-4.886,33.41,1.70,272.00,824.536,0.000,0.0,0.0000,7,52.08
2402,2402,301,541,490.631000,996.863000,168.59312,14.500,3.446,-13.654,11.30,2.45,154.90,811.310,0.000,0.0,0.2381,3,75.61
2403,2403,97,6,-0.957105,-0.143412,1.93913,6.984,2.890,-1.610,53.41,2.70,22.43,820.148,0.000,0.0,0.0000,3,108.22
2404,2404,186,558,502.869000,978.244000,174.94429,4.311,-1.305,-10.505,29.84,8.70,287.10,814.018,0.000,0.0,0.6220,7,9.83


In [9]:
x_test = df_xtest.iloc[:,1:18]
x_test

,DATE (YYYY/MM/DD),MST,Global CMP22 (vent/cor) [W/m^2],Direct sNIP [W/m^2],Azimuth Angle [degrees],Tower Dry Bulb Temp [deg C],Tower Wet Bulb Temp [deg C],Tower Dew Point Temp [deg C],Tower RH [%],Peak Wind Speed @ 6ft [m/s],Avg Wind Direction @ 6ft [deg from N],Station Pressure [mBar],Precipitation (Accumulated) [mm],Moisture,Albedo (CMP11),Cloud Type,Solar Zenith Angle
0,255,974,-0.821587,-0.197822,270.39668,0.344,-3.269,-9.469,43.43,1.95,300.70,817.643,0.000,0.0,0.0000,7,163.53
1,47,203,-0.464099,0.039564,83.18453,-15.420,-16.249,-19.649,67.07,1.20,308.30,809.469,0.000,0.0,0.0000,0,84.09
2,63,119,-1.133330,-0.009875,35.33772,22.380,11.522,3.122,28.25,1.70,214.00,813.392,0.000,0.0,0.0000,7,109.24
3,110,694,849.620000,946.128000,244.34737,14.240,6.585,-1.215,34.02,3.45,45.53,821.467,0.000,0.0,0.1844,7,156.80
4,127,765,44.193400,0.488831,230.89280,-5.580,-6.169,-7.269,86.40,1.20,15.69,812.324,0.000,1.0,0.6648,7,154.89
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2401,185,1040,-1.444170,-0.360447,292.96778,9.820,3.214,-4.886,33.41,1.70,272.00,824.536,0.000,0.0,0.0000,7,52.08
2402,301,541,490.631000,996.863000,168.59312,14.500,3.446,-13.654,11.30,2.45,154.90,811.310,0.000,0.0,0.2381,3,75.61
2403,97,6,-0.957105,-0.143412,1.93913,6.984,2.890,-1.610,53.41,2.70,22.43,820.148,0.000,0.0,0.0000,3,108.22
2404,186,558,502.869000,978.244000,174.94429,4.311,-1.305,-10.505,29.84,8.70,287.10,814.018,0.000,0.0,0.6220,7,9.83


In [10]:
model = linear_model.LinearRegression()
model.fit(x_train,y_train)

LinearRegression()

In [11]:
from sklearn.metrics import r2_score

# Predicting R2 Score the Train set results
y_pred_linear_train = model.predict(x_train)
r2_score_linear_train = r2_score(y_train, y_pred_linear_train)

print('R2_score (train): ', r2_score_linear_train)

R2_score (train):  0.009666725680287591


In [12]:
from sklearn.preprocessing import PolynomialFeatures
poly_reg = PolynomialFeatures(degree = 2)  #Make the instance of the Model of degree 2
X_poly = poly_reg.fit_transform(x_train)   #Fit on training set
poly_reg.fit(X_poly, y_train)
regressor_poly2 = LinearRegression()
regressor_poly2.fit(X_poly, y_train)

LinearRegression()

In [13]:
from sklearn.metrics import r2_score

# Predicting R2 Score the Train set results
y_pred_poly2_train = regressor_poly2.predict(poly_reg.fit_transform(x_train))
r2_score_poly2_train = r2_score(y_train, y_pred_poly2_train)

#calculate the Root mean Square Error
rmse_poly2 = (np.sqrt(mean_squared_error(y_train, y_pred_poly2_train)))  #(np.sqrt(mean_squared_error(y_test, y_pred_poly2_test)))
print('R2_score (train): ', r2_score_poly2_train)


R2_score (train):  0.07735195590713562


In [14]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
sc_y = StandardScaler()
X_scaled = sc_X.fit_transform(x_train)
y_scaled = sc_y.fit_transform(y_train)

In [15]:
# Fitting the SVR Model to the dataset
from sklearn.svm import SVR
regressor_svr = SVR(kernel = 'rbf', gamma = 'scale')
regressor_svr.fit(X_scaled, y_scaled.ravel())

SVR()

In [16]:
from sklearn.metrics import r2_score

# Predicting R2 Score the Train set results
y_pred_svr_train = sc_y.inverse_transform(regressor_svr.predict(sc_X.transform(x_train)).reshape(-1,1))
r2_score_svr_train = r2_score(y_train, y_pred_svr_train)

print('R2_score (train): ', r2_score_svr_train)

R2_score (train):  0.010103658208183264


In [17]:
models = [('Linear Regression', r2_score_linear_train),
          ('Polynomial Regression (2nd)', r2_score_poly2_train),
          ('Support Vector Regression', r2_score_svr_train),
         ]

In [18]:
predict = pd.DataFrame(data = models, columns=['Model', 'R2_Score(training)'])
predict

,Model,R2_Score(training)
0,Linear Regression,0.009667
1,Polynomial Regression (2nd),0.077352
2,Support Vector Regression,0.010104


In [19]:
#From above we see that polynomial regression has the best R2-score.
#Hence we use polynomial regression to create y_test data.
y_pred = regressor_poly2.predict(poly_reg.fit_transform(x_test))

In [20]:
y_pred = pd.DataFrame(y_pred)

In [30]:
y_pred.to_csv('Predicted_Snow_Depth.csv')
y_pred.columns = ['Snow Depth [cm]']
y_pred.index.name='ID'
y_pred

,Snow Depth [cm]
ID,
0,-66.697306
1,23.435054
2,45.026346
3,-59.320764
4,50.695560
...,...
2401,-37.464131
2402,-40.607249
2403,-33.156565
